In [103]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime
import mysql.connector

In [104]:
#function that will return soup from a url
def get_soup(url="https://stocktwits.com/rankings/watchers"):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.set_page_load_timeout(100)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def get_data_list(soup):
    # get All of the table row elements
    table_rows = soup.find_all('tr', class_='st_PaKabGw st_1jzr122')
    # for each row get ticker, count of new watches in 24 hours and the price move in 24 hours
    output_data = []
    for i in table_rows:
        ticker = i.find('span', class_='st_3ua_jwB st_8u0ePN3 st_1SuHTwr') # get ticker by class 
        count = i.find_all('td') # get count of watches as first td tag
        price_move = i.find_all('span')[-1] # get price move as the last span 
        if len(price_move['class'])<4:
            sign = 0
        elif price_move['class'][3] == 'st_3ftxMPa': # identify if pos or neg based on class
            sign = 1
        elif price_move['class'][3] == 'st_16rsg3e':
            sign = -1
        strip_price_move = float(price_move.text.rstrip('%'))
        price_move = sign * strip_price_move
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
#         print(now, " ", ticker.text, " ", count[3].text," ",price_move)
        output_data.append([now,ticker.text, count[3].text, price_move])
    return output_data

def get_cursor():
    mydb = mysql.connector.connect(
      host="35.246.42.55",
      user="root",
      password="RF/o<_4eyHm`>&^e"
    )
    mycursor = mydb.cursor()
    mycursor.execute("use stocks")
    return mycursor, mydb

def select_all(mycursor, mydb):
    mycursor.execute("use stocks")
    mycursor.execute("show tables")
    print(mycursor.fetchall())
    mycursor.execute("select * from stocknote")
    print(mycursor.fetchall())
    print('shown all rows')
    
def drop_all(mycursor, mydb):
    mycursor.execute("use stocks")
    mycursor.execute("delete from stocknote")
    mydb.commit()
    print('deleted all rows')
    

def insert_rows(data, cursor, connection):
    for i in data:
        mySql_insert_query = """INSERT INTO stocknote (datetime,ticker,watchers,pricechange) 
                            VALUES (%s, %s, %s, %s) """

        record = (i[0], i[1], i[2], i[3])
        cursor.execute(mySql_insert_query, record)
        connection.commit()
        print("Record inserted successfully into stocknote table")


In [78]:
def main():

    soup = get_soup()
    data = get_data_list(soup)
    cursor, db = get_cursor()
    select_all(cursor, db)
    drop_all(cursor, db)
    insert_rows(data, cursor, db)

    return
    
data = main()

/var/folders/5c/p41fpsts3xx71rmzxs3twj_h0000gn/T/ipykernel_38798/1179943068.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


[['19/08/2022 16:59:26', 'GCT', '2958', 193.18], ['19/08/2022 16:59:26', 'BBBY', '1881', -37.79], ['19/08/2022 16:59:26', 'MNMD', '1243', 0.98], ['19/08/2022 16:59:26', 'WEBR', '940', -11.0], ['19/08/2022 16:59:26', 'SMMT', '517', 14.0], ['19/08/2022 16:59:26', 'DRUG', '481', -21.2], ['19/08/2022 16:59:26', 'AAPL', '394', -1.11], ['19/08/2022 16:59:26', 'INDO', '386', -4.34], ['19/08/2022 16:59:26', 'TSLA', '347', -2.8], ['19/08/2022 16:59:26', 'AMZN', '330', -2.71]]
[('stocknote',)]
[('now', ' AAPL', '10', '-100')]
['19/08/2022 16:59:26', 'GCT', '2958', 193.18]
['19/08/2022 16:59:26', 'BBBY', '1881', -37.79]
['19/08/2022 16:59:26', 'MNMD', '1243', 0.98]
['19/08/2022 16:59:26', 'WEBR', '940', -11.0]
['19/08/2022 16:59:26', 'SMMT', '517', 14.0]
['19/08/2022 16:59:26', 'DRUG', '481', -21.2]
['19/08/2022 16:59:26', 'AAPL', '394', -1.11]
['19/08/2022 16:59:26', 'INDO', '386', -4.34]
['19/08/2022 16:59:26', 'TSLA', '347', -2.8]
['19/08/2022 16:59:26', 'AMZN', '330', -2.71]
(19/08/2022 16:59

In [114]:
#soup = get_soup()
#data = get_data_list(soup)
#cursor, db = get_cursor()
select_all(cursor, db)
#drop_all(cursor, db)
#insert_rows(data, cursor, db)

[('stocknote',)]
[('19/08/2022 17:17:50', 'GCT', '3028', '177.31'), ('19/08/2022 17:17:50', 'BBBY', '1866', '-35.42'), ('19/08/2022 17:17:50', 'MNMD', '1163', '2.45'), ('19/08/2022 17:17:50', 'WEBR', '930', '-8.5'), ('19/08/2022 17:17:50', 'SMMT', '506', '17.0'), ('19/08/2022 17:17:50', 'AAPL', '392', '-1.13'), ('19/08/2022 17:17:50', 'TSLA', '350', '-2.7'), ('19/08/2022 17:17:50', 'BTC-X', '336', '-8.07'), ('19/08/2022 17:17:50', 'AMZN', '322', '-2.54'), ('19/08/2022 17:17:50', 'SPY', '311', '-1.19')]
shown all rows
